In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import time

In [ ]:
def setup_browser():
    options = uc.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-infobars')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.6834.160 Safari/537.36')
    
    browser = uc.Chrome(
        options=options,
        version_main=132
    )
    return browser

def collect_links(browser, target_count=10000):
    link = []
    while True:
        try:
            page_source = browser.execute_script("return document.documentElement.outerHTML")
            soup = BeautifulSoup(page_source, 'html.parser')
            
            for soup1 in soup.find_all('div', class_='x1lliihq x1n2onr6 xh8yej3 x4gyw5p x1ntc13c x9i3mqj x11i5rnm x2pgyrj'):
                superlink = soup1.find('a', class_='x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _a6hd')
                
                if superlink:
                    href_link = superlink.get('href')
                    if 'muse' in href_link and 'followers' not in href_link:
                        linkss = 'https://www.instagram.com' + href_link
                        if linkss not in link:
                            link.append(linkss)
                            print(f"找到的連結 ({len(link)}): {linkss}")
            
            # 滾動頁面
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 增加等待時間以確保內容加載
            
            if len(link) >= target_count:
                break
                
        except Exception as e:
            print(f"收集連結時發生錯誤: {str(e)}")
            break
            
    return link

def analyze_posts(browser, links, keywords):
    filtered_posts = []
    posts = []
    keywords = ['泰國','越南','印尼']

    for index, link in enumerate(links, 1):
        try:
            print(f"處理第 {index}/{len(links)} 個連結")
            browser.get(link)
            time.sleep(5)
            
            page_source = browser.execute_script("return document.documentElement.outerHTML")
            soup = BeautifulSoup(page_source, 'html.parser')
            
            for context in soup.find_all('h1', class_='_ap3a *aaco *aacu *aacx *aad7 _aade'):
                post_text = context.text
                posts.append(post_text)
                
                if any(keyword in post_text for keyword in keywords):
                    filtered_posts.append(post_text)
                    print(f"找到符合關鍵字的貼文: {post_text}")
                    
        except Exception as e:
            print(f"分析貼文時發生錯誤: {str(e)}")
            continue
            
    return posts, filtered_posts

def main():
    browser = None
    try:
        browser = setup_browser()
        print("瀏覽器創建成功")
        
        # 訪問Instagram
        browser.get('https://www.instagram.com/museacg/')
        print("頁面訪問成功")
        
        # 登入流程
        time.sleep(5)
        login_element = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), '登入')]"))
        )
        login_element.click()
        print("點擊登入按鈕")
        
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.NAME, "username"))
        )
        
        username = browser.find_element(By.NAME, 'username')
        password = browser.find_element(By.NAME, "password")
        
        username.send_keys("在此輸入個人帳號")
        password.send_keys("在此輸入個人密碼")
        print("輸入帳號密碼")
        
        browser.find_element(By.XPATH, '//button[@type="submit"]').click()
        print("點擊登入按鈕")
        
        time.sleep(5)
        
        # 重新訪問目標頁面
        browser.get('https://www.instagram.com/museacg/')
        print("重新訪問頁面成功")
        
        time.sleep(5)
        
        # 收集連結
        print("開始收集連結...")
        links = collect_links(browser)
        print(f"總共收集到 {len(links)} 個連結")
        
        # 分析貼文
        print("開始分析貼文...")
        keywords = ['泰國', '越南', '印尼']
        posts, filtered_posts = analyze_posts(browser, links, keywords)
        
        print(f"分析完成！總共找到 {len(filtered_posts)} 個符合關鍵字的貼文")
        
    except Exception as e:
        print(f"發生錯誤: {str(e)}")
    
    finally:
        if browser:
            browser.quit()
            print("瀏覽器已關閉")

if __name__ == "__main__":
    main()